<a href="https://colab.research.google.com/github/kaitz2981-ux/TTNT-Tuan03/blob/main/Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bài 1

    import math

    Player_X = 'X'
    Player_O = 'O'
    EMPTY = ' '
    BOARD_SIZE = 3

    class TicTacToe
    def __init__(self):
    self.board = [[EMPTY for_in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

#Kiểm tra xem người chơi 'player' đã thắng chưa
  # 1. Kiểm tra hàng
    for row in range(BOARD_SIZE):
    if all(self.board[row][col] == player for col in range(BOARD_SIZE)):
    return True
        
# 2. Kiểm tra cột
    for col in range(BOARD_SIZE):
    if all(self.board[row][col] == player for row in range(BOARD_SIZE)):
    return True
        
# 3. Kiểm tra đường chéo chính
    if all(self.board[i][i] == player for i in range(BOARD_SIZE)):
    return True
        
# 4. Kiểm tra đường chéo phụ
    if all(self.board[i][BOARD_SIZE - 1 - i] == player for i in range(BOARD_SIZE)):
    return True
            
    return False

    def _is_moves_left(self):

#Kiểm tra xem còn ô trống nào trên bảng không
    for row in range(BOARD_SIZE):
    for col in range(BOARD_SIZE):
    if self.board[row][col] == EMPTY:
    return True
    return False

    def evaluate(self):